In [1]:
import pandas as pd

In [12]:
def cross_check_records(check_video_type = None, input_file = "../table_tennis_data/metadata/game.xlsx"):
    # Read Excel sheets
    df_game = pd.read_excel(input_file, sheet_name="game")
    df_game_player = pd.read_excel(input_file, sheet_name="game_player")
    df_game_player_item = pd.read_excel(input_file, sheet_name="game_player_item")
    df_game_player_double_item = pd.read_excel(input_file, sheet_name="game_player_double_item")

    # Ensure required columns exist
    required_cols = {
        "game": ["Id"],
        "game_player": ["GameId", "PlayerId"],
        "game_player_item": ["GamePlayerId"],
        "game_player_double_item": ["GamePlayerId"],
    }

    for name, cols in required_cols.items():
        df = {
            "game": df_game,
            "game_player": df_game_player,
            "game_player_item": df_game_player_item,
            "game_player_double_item": df_game_player_double_item,
        }[name]

        for col in cols:
            if col not in df.columns:
                raise ValueError(f"Missing column '{col}' in {name} file.")

    # Convert gameplayerid to set for fast lookup
    item_ids = set(df_game_player_item["GamePlayerId"].tolist()) | set(df_game_player_double_item["GamePlayerId"].tolist())

    results = []

    # to get the non-player missing game ids
    non_missing_game_ids = []

    # Process each game.id
    for _, row in df_game.iterrows():

        game_id = row["Id"]
        video_type = row["VideoType"]
        if check_video_type:
            if video_type != video_type:
                continue

        # Step 1: Find all players for this game
        players = df_game_player[df_game_player["GameId"] == game_id]["PlayerId"].tolist()

        # Step 2: Check each player_id against game_player_item
        missing = [pid for pid in players if pid not in item_ids]
        existing = [pid for pid in players if pid in item_ids]
        if not missing: non_missing_game_ids.append(game_id)

        results.append({
            "game_id": game_id,
            "total_players": len(players),
            "players_found_in_item": existing,
            "players_missing_in_item": missing
        })

    return results, non_missing_game_ids

In [15]:
# results, non_missing_game_ids = cross_check_records("File")
# for r in results:
#     print("\nGame ID:", r["game_id"])
#     print("Found players:", r["players_found_in_item"])
#     print("Missing players:", r["players_missing_in_item"])

In [16]:
# print(non_missing_game_ids)